<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/instagramm_pars_07022019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Получение фото

In [0]:
import hashlib
import json
import re
import requests


count=36
session = requests.Session()

      
def update(obj=None, pointer=None, idx=None, rhx_gis=None, settings={}):

      src_adress = [] 
      base_url = ""
      query = "https://www.instagram.com/"
      if not obj is None:
            query += base_url + obj
            
        
      if pointer is None: 
        response = session.get(query, **settings)
        match = re.search(r"<script[^>]*>\s*window._sharedData\s*=\s*((?!<script>).*)\s*;\s*</script>", response.text)
        data = json.loads(match.group(1))
        rhx_gis = data["rhx_gis"]
        csrf_token = data["config"]["csrf_token"]
        idx = data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["id"]
        
        #print("end_cursor", data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor'])
        #print("'has_next_page", data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page'])
        
        for number in range(len(data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]["edges"])):
	          src_adress.append(data["entry_data"]["ProfilePage"][0]["graphql"]["user"]["edge_owner_to_timeline_media"]["edges"][number]["node"]["thumbnail_resources"][4]["src"])
        if data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page']:                 
            pointer = data['entry_data']['ProfilePage'][0]['graphql']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor']
        else:
            pointer = None

        return rhx_gis, pointer, src_adress, idx, csrf_token

      
      else:
        variables_string = '{{"id":"{id}","first":{first},"after":"{after}"}}'
        while True:
            data = {"id": idx, "first": 36, "after": pointer}
            variables=variables_string.format(**data)
            query_hash="c6809c9c025875ac6f02619eae97a80e"
            #response = graphql_request(query_hash="c6809c9c025875ac6f02619eae97a80e", variables=variables, settings=settings)
            settings["params"] = {"query_hash": query_hash}
            settings["params"]["variables"] = variables
            gis = "%s:%s" % (rhx_gis, variables)
            settings["headers"] = {"X-Instagram-GIS": hashlib.md5(gis.encode("utf-8")).hexdigest()}
            settings["headers"]["X-Requested-With"] = "XMLHttpRequest" 
            response = session.get("https://www.instagram.com/graphql/query/", **settings)
            response.raise_for_status()
            
            data = response.json()["data"]["user"]["edge_owner_to_timeline_media"]

            edges = data["edges"]
            page_info = data["page_info"]
            length = len(edges)

            for index in range(length):
                    node = edges[index]["node"]
                    shortcode = node["shortcode"]        #["src"]
                    if not "shortcode" in node:
                        length -= 1
                        continue
                    m = node["display_url"]
                    #m._set_data(node)
                    src_adress.append(m)
                    #print("node", node)
                
            if page_info["has_next_page"]:
                    
                    pointer = page_info["end_cursor"]
            else:
                    pointer = None

            return src_adress, pointer

      

In [0]:
src_adress = []
rhx_gis, pointer, src, idx, csrf_token = update("likekate_")
src_adress.extend(src)
print("rhx_gis:", rhx_gis, "csrf_token:", csrf_token, "idx =", idx)
for i in range(100):
  src, pointer = update("likekate_", pointer=pointer, idx=idx, rhx_gis=rhx_gis)
  src_adress.extend(src)
  print(len(src_adress))


# Авторизация

In [3]:
import hashlib
import json
import re
import requests
session = requests.Session()
# rhx_gis: fc2e73d4fd7dddcd31d28bea5cb2df59 csrf_token: 23QtJMvu9FzAuskGBKqVy3PcPvOdvxnv
def AgentAccount(login=None, password=None, settings={}, load=False):
    # load cookie    
        if load:
           print("1")
           with open('cookie.txt', 'r') as f:
              cookie = requests.utils.cookiejar_from_dict(json.load(f))
              print(cookie)
              session.hooks['response'].append(cookie)
              response = session.get("https://www.instagram.com/")
              response.raise_for_status()
              print("response1:", response)
        else:
            print("2")
            csrf_token = "23QtJMvu9FzAuskGBKqVy3PcPvOdvxnv"
            login = "andivanov2019"
            password = "Asus2046"
            data = {"username": login, "password": password}
        
            if "headers" in settings:
                #settings["params"] = {"query_hash": query_hash}
                settings["headers"]["X-CSRFToken"] = csrf_token
                settings["headers"]["referer"] = "https://www.instagram.com/"
            else:
                settings["headers"] = {
                    "X-CSRFToken": csrf_token,
                    "referer": "https://www.instagram.com/",
                       }

            response = session.post("https://www.instagram.com/accounts/login/ajax/", data, **settings)
            response.raise_for_status()
            # Save cookie
            with open('cookie.txt', 'w') as f:
               json.dump(requests.utils.dict_from_cookiejar(response.cookies), f)
            print("response2:", response)
            try:
               data = response.json()
               if not data["authenticated"]:
                  raise AuthException(login) 
            except (ValueError, KeyError) as exception:
               print("ERROR1")

AgentAccount(load=True)
#AgentAccount()

1
<RequestsCookieJar[<Cookie csrftoken=F6PUydMLF4oX3C7xmb7pAuooKjuXoq3D for />, <Cookie ds_user_id=10397645874 for />, <Cookie mid=XFw5ZgAEAAFC31Iuks5751j72zx_ for />, <Cookie rur=FTW for />, <Cookie sessionid=10397645874%3A1LioPJUFSTz0wI%3A3 for />, <Cookie shbid=2881 for />, <Cookie shbts=1549547879.181792 for />]>
response1: <Response [200]>


In [7]:
import os
#session.close()
#response.raise_for_status()
#session
print( os.listdir('/content') )
from google.colab import files
files.download('cookie.txt') 

['.config', 'cookie.txt', 'sample_data']


In [2]:
from google.colab import files
files.upload()

Saving cookie.txt to cookie.txt


{'cookie.txt': b'{"csrftoken": "F6PUydMLF4oX3C7xmb7pAuooKjuXoq3D", "ds_user_id": "10397645874", "mid": "XFw5ZgAEAAFC31Iuks5751j72zx_", "rur": "FTW", "sessionid": "10397645874%3A1LioPJUFSTz0wI%3A3", "shbid": "2881", "shbts": "1549547879.181792"}'}

In [1]:
import os
print( os.listdir('/content') )

['.config', 'sample_data']


# Получение подписчиков


In [4]:
#query_hash:56066f031e6239f35a904ac20c9f37d9
#variables:{"id":"10180157718","include_reel":true,"fetch_mutual":false,"first":12,"after":""}

import hashlib
from time import sleep
from requests.exceptions import HTTPError
def get_followers(account=None, pointer=None, count=100000, settings={}, limit=50):
  
        #update(account, settings)
        csrf_token = "23QtJMvu9FzAuskGBKqVy3PcPvOdvxnv"
        rhx_gis = "fc2e73d4fd7dddcd31d28bea5cb2df59"
        #query_hash = "QVFENklrNmY2Y2JvRGhEa2k5UV9XZTVkRHcwSDB6OXM3dWhhZkVsSXpNdlZacEJQVU9QVkcybDQtaC11c04tSXN3cWlmUzVIN3NxeG1BR2phV0VYSU1fcw=="
        query_hash="37479f2b8209594dde7facb0d904896a"
        idx = "2120762022"

        if pointer is None:
            #variables_string = '{{"id":"{id}","include_reel":true,"fetch_mutual":false,"first":{first}}}'
            variables_string = '{{"id":"{id}","first":{first}}}'
        else:
            #variables_string = '{{"id":"{id}","include_reel":true,"fetch_mutual":false,"first":{first},"after":"{after}"}}'
            variables_string = '{{"id":"{id}","first":{first},"after":"{after}"}}'
        followers = []

        while True:
            data = {"first": min(limit, count), "id": idx}
            if not pointer is None:
                data["after"] = pointer
            settings["params"] = {"query_hash": query_hash}
            variables=variables_string.format(**data)
            settings["params"]["variables"] = variables
            gis = "%s:%s" % (rhx_gis, variables)
            settings["headers"] = {"X-Instagram-GIS": hashlib.md5(gis.encode("utf-8")).hexdigest()}
            settings["headers"]["X-Requested-With"] = "XMLHttpRequest" 
            #response = session.get("https://www.instagram.com/graphql/query/", **settings)
            #response.raise_for_status()
            
            
           
            for i in range(0, 3):
              try:
                response = session.get("https://www.instagram.com/graphql/query/", **settings)
                print(response.headers)
                response.raise_for_status()

              except (HTTPError, ValueError, KeyError) as exception:

                    print("ERROR", exception.response.status_code)
  
                    print("sleep") 
                    sleep(6) 
                    print("end sleep")
                    continue
              break
            #data = response.json()
            data = response.json()["data"]["user"]["edge_followed_by"]
            edges = data["edges"]
            page_info = data["page_info"]
            #print(page_info)
            followers_count = data["count"]
                
            for index in range(min(len(edges), count)):
                    node = edges[index]["node"]
                    a = node["username"]
                    id = node["id"]
                    profile_pic_url = node["profile_pic_url"]
                    is_verified = node["is_verified"]
                    full_name = node["full_name"]
                    followers.append(a)
                    if len(followers) % 100 == 0:
                      print(len(followers))
                
            if page_info["has_next_page"]:
                    pointer = page_info["end_cursor"]
            else:
                    pointer = None
                
            if len(edges) < count and page_info["has_next_page"]:
                    count = count-len(edges)
                    variables_string = '{{"id":"{id}","first":{first},"after":"{after}"}}'
            else:
                    return followers, pointer
            
                
get_followers("likekate_")

{'Content-Type': 'application/json; charset=utf-8', 'Vary': 'Accept-Language, Cookie', 'Content-Language': 'en', 'Date': 'Thu, 07 Feb 2019 13:59:39 GMT', 'Strict-Transport-Security': 'max-age=604800', 'Cache-Control': 'private, no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Expires': 'Sat, 01 Jan 2000 00:00:00 GMT', 'X-Frame-Options': 'SAMEORIGIN', 'content-security-policy': "report-uri https://www.instagram.com/security/csp_report/; default-src 'self' https://www.instagram.com; img-src https: data: blob:; font-src https: data:; media-src 'self' blob: https://www.instagram.com https://*.cdninstagram.com https://*.fbcdn.net; manifest-src 'self' https://www.instagram.com; script-src 'self' https://instagram.com https://www.instagram.com https://*.www.instagram.com https://*.cdninstagram.com wss://www.instagram.com https://*.facebook.com https://*.fbcdn.net https://*.facebook.net 'unsafe-inline' 'unsafe-eval' blob:; style-src 'self' https://*.www.instagram.com https://www.in

([], None)

In [0]:
followers

NameError: ignored

In [13]:
get_followers("likekate_", pointer="QVFDSGVsXzh6MjVkWW5Oc1dNQ1RVcFlkOU5WTDFBcHVoMGozQXpGVjJvU1Q3M1RPc3lhMDRwaS1SYm9uVjdFY2dkZmU1eWtScUZ6cXc3YURMT2k3RnZkZw==")

{'Content-Type': 'application/json; charset=utf-8', 'Vary': 'Accept-Language, Cookie', 'Content-Language': 'en', 'Date': 'Thu, 07 Feb 2019 13:48:28 GMT', 'Strict-Transport-Security': 'max-age=604800', 'Cache-Control': 'private, no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Expires': 'Sat, 01 Jan 2000 00:00:00 GMT', 'X-Frame-Options': 'SAMEORIGIN', 'content-security-policy': "report-uri https://www.instagram.com/security/csp_report/; default-src 'self' https://www.instagram.com; img-src https: data: blob:; font-src https: data:; media-src 'self' blob: https://www.instagram.com https://*.cdninstagram.com https://*.fbcdn.net; manifest-src 'self' https://www.instagram.com; script-src 'self' https://instagram.com https://www.instagram.com https://*.www.instagram.com https://*.cdninstagram.com wss://www.instagram.com https://*.facebook.com https://*.fbcdn.net https://*.facebook.net 'unsafe-inline' 'unsafe-eval' blob:; style-src 'self' https://*.www.instagram.com https://www.in

([], None)